In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import scipy.stats as st
import math
import statistics


# Import API Keys
from api_keys import walk_key


In [2]:
file = "Resources/sales+school.csv"
df = pd.read_csv(file)
df = df.rename(columns = {"Total Sq. Ft.":"Total Sq Ft"})
df["dollar/sqft"] = (df["Sale Price"] / df["Total Sq Ft"]).astype(float)
df

,MLS ID,Sale Price,Close Date,Address,Zip Code,Total Sq Ft,Lot Size,Latitude,Longitude,High School,High School Rank,Middle School,Middle School Rank,Elementary School,Elementary School Rank,dollar/sqft
0,2001944,240000.0,2021-03-15,4917 BlueStem DR,80917,1170.0,711.0,38.8976243,-104.737074,Doherty High School,2.0,Russell Middle School,2.0,Carver Elementary School,1.0,205.128205
1,8359484,237500.0,2021-03-12,54 Villa DR,81001,2087.0,6509.0,38.8654252,-104.863362,Coronado High School,3.0,Holmes Middle School,3.0,Howbert Elementary School,3.0,113.799713
2,7312436,275000.0,2021-03-15,132 Hayden ST,80813,2503.0,9583.0,38.750711,-105.176383,Cripple Creek-Victor Junior-Senior High School,1.0,NaN,NaN,Cresson Elementary School,1.0,109.868158
3,1398523,566455.0,2021-03-18,6596 Wolf Gulch DR,80924,4714.0,9088.0,38.9603997,-104.703074,Liberty High School,4.0,Chinook Trail Middle School,NaN,Legacy Peak Elementary School,3.0,120.164404
4,4306524,410000.0,2021-03-12,6562 Tillamook DR,80925,1949.0,6600.0,38.8338816,-104.821363,Palmer High School,3.0,NaN,2.0,Coperni 2,NaN,210.364289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349,4628075,890000.0,2021-04-12,16 Polo DR,80906,3063.0,10890.0,38.7974702,-104.841016,Palmer High School,5.0,NaN,5.0,NaN,NaN,290.564806
4350,5664354,915000.0,2021-04-12,17925 Black Squirrel RD,80908,4808.0,625086.0,39.092056,-104.627332,Smoky Hill High School,NaN,NaN,NaN,Talbott Steam Innovation School,NaN,190.307820
4351,1719141,925000.0,2021-04-09,390 Paisley DR,80906,4822.0,13068.0,38.7435755,-104.835224,Mitchell High School,2.0,NaN,NaN,Stratmoor Hills Elementary School,5.0,191.829117
4352,3250460,990900.0,2021-04-15,4937 Rainbow Gulch TR,80924,6140.0,12120.0,38.9758208,-104.735368,Liberty High School,4.0,NaN,NaN,Pinello Elementary School,5.0,161.384365


In [3]:
# Making test call
url = f"https://api.walkscore.com/score?format=json&lat=38.8976243&lon=-104.737074&transit=1&bike=1&wsapikey={walk_key}"

response = requests.get(url)

response_json = response.json()
print(response.status_code)
print(json.dumps(response_json, indent=4))
print(response_json['walkscore'])

200
{
    "status": 1,
    "walkscore": 53,
    "description": "Somewhat Walkable",
    "updated": "2021-03-31 22:09:56.538152",
    "logo_url": "https://cdn.walk.sc/images/api-logo.png",
    "more_info_icon": "https://cdn.walk.sc/images/api-more-info.gif",
    "more_info_link": "https://www.redfin.com/how-walk-score-works",
    "ws_link": "https://www.walkscore.com/score/loc/lat=38.8976243/lng=-104.737074/?utm_source=robthompsonrealtor.com&utm_medium=ws_api&utm_campaign=ws_api",
    "help_link": "https://www.redfin.com/how-walk-score-works",
    "snapped_lat": 38.898,
    "snapped_lon": -104.7375,
    "transit": {
        "score": null,
        "description": null,
        "summary": null
    },
    "bike": {
        "score": 53,
        "description": "Bikeable"
    }
}
53


In [4]:
# Doing actual API to retrieve walk score
walkscore = []
errors= []

count = 1
errorcount = 0
linebreak = "-----------------------------------------------------------------------------------------"

print("Beginning Walk Score Retrieval")
print(linebreak)

for index, row in df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    
    url=f"https://api.walkscore.com/score?format=json&lat={lat}&lon={lon}&transit=1&bike=1&wsapikey={walk_key}"
    
    try:
        walkresponse = requests.get(url)
        walkjson = walkresponse.json()
        walkscore.append(walkjson['walkscore'])
        print(f"Retrieved Score #{count} of {len(df)}")
        count += 1
        
    except:
        print(f"Error #{errorcount+1}")
        walkscore.append('NaN')
        errors.append(errorcount)
        count += 1
        errorcount += 1
        pass

print(linebreak)
print("Finished with Retrieval")
print(f"{len(walkscore)} walkscores were found")
print(f"{errorcount+1} Errors were found")
print(errors)


Beginning Walk Score Retrieval
-----------------------------------------------------------------------------------------
Retrieved Score #1 of 4354
Retrieved Score #2 of 4354
Retrieved Score #3 of 4354
Retrieved Score #4 of 4354
Retrieved Score #5 of 4354
Retrieved Score #6 of 4354
Retrieved Score #7 of 4354
Retrieved Score #8 of 4354
Retrieved Score #9 of 4354
Retrieved Score #10 of 4354
Retrieved Score #11 of 4354
Retrieved Score #12 of 4354
Retrieved Score #13 of 4354
Retrieved Score #14 of 4354
Retrieved Score #15 of 4354
Retrieved Score #16 of 4354
Retrieved Score #17 of 4354
Retrieved Score #18 of 4354
Retrieved Score #19 of 4354
Retrieved Score #20 of 4354
Retrieved Score #21 of 4354
Retrieved Score #22 of 4354
Retrieved Score #23 of 4354
Retrieved Score #24 of 4354
Retrieved Score #25 of 4354
Retrieved Score #26 of 4354
Retrieved Score #27 of 4354
Retrieved Score #28 of 4354
Retrieved Score #29 of 4354
Retrieved Score #30 of 4354
Retrieved Score #31 of 4354
Retrieved Score #32 

Retrieved Score #284 of 4354
Retrieved Score #285 of 4354
Retrieved Score #286 of 4354
Retrieved Score #287 of 4354
Retrieved Score #288 of 4354
Retrieved Score #289 of 4354
Retrieved Score #290 of 4354
Retrieved Score #291 of 4354
Retrieved Score #292 of 4354
Retrieved Score #293 of 4354
Retrieved Score #294 of 4354
Retrieved Score #295 of 4354
Retrieved Score #296 of 4354
Retrieved Score #297 of 4354
Retrieved Score #298 of 4354
Retrieved Score #299 of 4354
Retrieved Score #300 of 4354
Retrieved Score #301 of 4354
Retrieved Score #302 of 4354
Retrieved Score #303 of 4354
Retrieved Score #304 of 4354
Retrieved Score #305 of 4354
Retrieved Score #306 of 4354
Retrieved Score #307 of 4354
Retrieved Score #308 of 4354
Retrieved Score #309 of 4354
Retrieved Score #310 of 4354
Retrieved Score #311 of 4354
Retrieved Score #312 of 4354
Retrieved Score #313 of 4354
Retrieved Score #314 of 4354
Retrieved Score #315 of 4354
Retrieved Score #316 of 4354
Retrieved Score #317 of 4354
Retrieved Scor

Retrieved Score #567 of 4354
Retrieved Score #568 of 4354
Retrieved Score #569 of 4354
Retrieved Score #570 of 4354
Retrieved Score #571 of 4354
Retrieved Score #572 of 4354
Retrieved Score #573 of 4354
Retrieved Score #574 of 4354
Retrieved Score #575 of 4354
Retrieved Score #576 of 4354
Retrieved Score #577 of 4354
Retrieved Score #578 of 4354
Retrieved Score #579 of 4354
Retrieved Score #580 of 4354
Retrieved Score #581 of 4354
Retrieved Score #582 of 4354
Retrieved Score #583 of 4354
Retrieved Score #584 of 4354
Retrieved Score #585 of 4354
Retrieved Score #586 of 4354
Retrieved Score #587 of 4354
Retrieved Score #588 of 4354
Retrieved Score #589 of 4354
Retrieved Score #590 of 4354
Retrieved Score #591 of 4354
Retrieved Score #592 of 4354
Retrieved Score #593 of 4354
Retrieved Score #594 of 4354
Retrieved Score #595 of 4354
Retrieved Score #596 of 4354
Retrieved Score #597 of 4354
Retrieved Score #598 of 4354
Retrieved Score #599 of 4354
Retrieved Score #600 of 4354
Retrieved Scor

Retrieved Score #850 of 4354
Retrieved Score #851 of 4354
Retrieved Score #852 of 4354
Retrieved Score #853 of 4354
Retrieved Score #854 of 4354
Retrieved Score #855 of 4354
Retrieved Score #856 of 4354
Retrieved Score #857 of 4354
Retrieved Score #858 of 4354
Retrieved Score #859 of 4354
Retrieved Score #860 of 4354
Retrieved Score #861 of 4354
Retrieved Score #862 of 4354
Retrieved Score #863 of 4354
Retrieved Score #864 of 4354
Retrieved Score #865 of 4354
Retrieved Score #866 of 4354
Retrieved Score #867 of 4354
Retrieved Score #868 of 4354
Retrieved Score #869 of 4354
Retrieved Score #870 of 4354
Retrieved Score #871 of 4354
Retrieved Score #872 of 4354
Retrieved Score #873 of 4354
Retrieved Score #874 of 4354
Retrieved Score #875 of 4354
Retrieved Score #876 of 4354
Retrieved Score #877 of 4354
Retrieved Score #878 of 4354
Retrieved Score #879 of 4354
Retrieved Score #880 of 4354
Retrieved Score #881 of 4354
Retrieved Score #882 of 4354
Retrieved Score #883 of 4354
Retrieved Scor

Retrieved Score #1129 of 4354
Retrieved Score #1130 of 4354
Retrieved Score #1131 of 4354
Retrieved Score #1132 of 4354
Retrieved Score #1133 of 4354
Retrieved Score #1134 of 4354
Retrieved Score #1135 of 4354
Retrieved Score #1136 of 4354
Retrieved Score #1137 of 4354
Retrieved Score #1138 of 4354
Retrieved Score #1139 of 4354
Retrieved Score #1140 of 4354
Retrieved Score #1141 of 4354
Retrieved Score #1142 of 4354
Retrieved Score #1143 of 4354
Retrieved Score #1144 of 4354
Retrieved Score #1145 of 4354
Retrieved Score #1146 of 4354
Retrieved Score #1147 of 4354
Retrieved Score #1148 of 4354
Retrieved Score #1149 of 4354
Retrieved Score #1150 of 4354
Retrieved Score #1151 of 4354
Retrieved Score #1152 of 4354
Retrieved Score #1153 of 4354
Retrieved Score #1154 of 4354
Retrieved Score #1155 of 4354
Retrieved Score #1156 of 4354
Retrieved Score #1157 of 4354
Retrieved Score #1158 of 4354
Retrieved Score #1159 of 4354
Retrieved Score #1160 of 4354
Retrieved Score #1161 of 4354
Retrieved 

...

Retrieved Score #1951 of 4354
Retrieved Score #1952 of 4354
Retrieved Score #1953 of 4354
Retrieved Score #1954 of 4354
Retrieved Score #1955 of 4354
Retrieved Score #1956 of 4354
Retrieved Score #1957 of 4354
Retrieved Score #1958 of 4354
Retrieved Score #1959 of 4354
Retrieved Score #1960 of 4354
Retrieved Score #1961 of 4354
Retrieved Score #1962 of 4354
Retrieved Score #1963 of 4354
Retrieved Score #1964 of 4354
Retrieved Score #1965 of 4354
Retrieved Score #1966 of 4354
Retrieved Score #1967 of 4354
Retrieved Score #1968 of 4354
Retrieved Score #1969 of 4354
Retrieved Score #1970 of 4354
Retrieved Score #1971 of 4354
Retrieved Score #1972 of 4354
Retrieved Score #1973 of 4354
Retrieved Score #1974 of 4354
Retrieved Score #1975 of 4354
Retrieved Score #1976 of 4354
Retrieved Score #1977 of 4354
Retrieved Score #1978 of 4354
Retrieved Score #1979 of 4354
Retrieved Score #1980 of 4354
Retrieved Score #1981 of 4354
Retrieved Score #1982 of 4354
Retrieved Score #1983 of 4354
Retrieved 

Retrieved Score #2225 of 4354
Retrieved Score #2226 of 4354
Retrieved Score #2227 of 4354
Retrieved Score #2228 of 4354
Retrieved Score #2229 of 4354
Retrieved Score #2230 of 4354
Retrieved Score #2231 of 4354
Retrieved Score #2232 of 4354
Retrieved Score #2233 of 4354
Retrieved Score #2234 of 4354
Retrieved Score #2235 of 4354
Retrieved Score #2236 of 4354
Retrieved Score #2237 of 4354
Retrieved Score #2238 of 4354
Retrieved Score #2239 of 4354
Retrieved Score #2240 of 4354
Retrieved Score #2241 of 4354
Retrieved Score #2242 of 4354
Retrieved Score #2243 of 4354
Retrieved Score #2244 of 4354
Retrieved Score #2245 of 4354
Retrieved Score #2246 of 4354
Retrieved Score #2247 of 4354
Retrieved Score #2248 of 4354
Retrieved Score #2249 of 4354
Retrieved Score #2250 of 4354
Retrieved Score #2251 of 4354
Retrieved Score #2252 of 4354
Retrieved Score #2253 of 4354
Retrieved Score #2254 of 4354
Retrieved Score #2255 of 4354
Retrieved Score #2256 of 4354
Retrieved Score #2257 of 4354
Retrieved 

Retrieved Score #2499 of 4354
Retrieved Score #2500 of 4354
Retrieved Score #2501 of 4354
Retrieved Score #2502 of 4354
Retrieved Score #2503 of 4354
Retrieved Score #2504 of 4354
Retrieved Score #2505 of 4354
Retrieved Score #2506 of 4354
Retrieved Score #2507 of 4354
Retrieved Score #2508 of 4354
Retrieved Score #2509 of 4354
Retrieved Score #2510 of 4354
Retrieved Score #2511 of 4354
Retrieved Score #2512 of 4354
Retrieved Score #2513 of 4354
Retrieved Score #2514 of 4354
Retrieved Score #2515 of 4354
Retrieved Score #2516 of 4354
Retrieved Score #2517 of 4354
Error #3
Retrieved Score #2519 of 4354
Retrieved Score #2520 of 4354
Retrieved Score #2521 of 4354
Retrieved Score #2522 of 4354
Retrieved Score #2523 of 4354
Retrieved Score #2524 of 4354
Retrieved Score #2525 of 4354
Retrieved Score #2526 of 4354
Retrieved Score #2527 of 4354
Retrieved Score #2528 of 4354
Retrieved Score #2529 of 4354
Retrieved Score #2530 of 4354
Retrieved Score #2531 of 4354
Retrieved Score #2532 of 4354
R

Error #408
Error #409
Error #410
Error #411
Error #412
Error #413
Error #414
Error #415
Error #416
Error #417
Error #418
Error #419
Error #420
Error #421
Error #422
Error #423
Error #424
Error #425
Error #426
Error #427
Error #428
Error #429
Error #430
Error #431
Error #432
Error #433
Error #434
Error #435
Error #436
Error #437
Error #438
Error #439
Error #440
Error #441
Error #442
Error #443
Error #444
Error #445
Error #446
Error #447
Error #448
Error #449
Error #450
Error #451
Error #452
Error #453
Error #454
Error #455
Error #456
Error #457
Error #458
Error #459
Error #460
Error #461
Error #462
Error #463
Error #464
Error #465
Error #466
Error #467
Error #468
Error #469
Error #470
Error #471
Error #472
Error #473
Error #474
Error #475
Error #476
Error #477
Error #478
Error #479
Error #480
Error #481
Error #482
Error #483
Error #484
Error #485
Error #486
Error #487
Error #488
Error #489
Error #490
Error #491
Error #492
Error #493
Error #494
Error #495
Error #496
Error #497
Error #498

Error #1143
Error #1144
Error #1145
Error #1146
Error #1147
Error #1148
Error #1149
Error #1150
Error #1151
Error #1152
Error #1153
Error #1154
Error #1155
Error #1156
Error #1157
Error #1158
Error #1159
Error #1160
Error #1161
Error #1162
Error #1163
Error #1164
Error #1165
Error #1166
Error #1167
Error #1168
Error #1169
Error #1170
Error #1171
Error #1172
Error #1173
Error #1174
Error #1175
Error #1176
Error #1177
Error #1178
Error #1179
Error #1180
Error #1181
Error #1182
Error #1183
Error #1184
Error #1185
Error #1186
Error #1187
Error #1188
Error #1189
Error #1190
Error #1191
Error #1192
Error #1193
Error #1194
Error #1195
Error #1196
Error #1197
Error #1198
Error #1199
Error #1200
Error #1201
Error #1202
Error #1203
Error #1204
Error #1205
Error #1206
Error #1207
Error #1208
Error #1209
Error #1210
Error #1211
Error #1212
Error #1213
Error #1214
Error #1215
Error #1216
Error #1217
Error #1218
Error #1219
Error #1220
Error #1221
Error #1222
Error #1223
Error #1224
Error #1225
Erro

In [5]:
df["Walk Score"] = walkscore
df

,MLS ID,Sale Price,Close Date,Address,Zip Code,Total Sq Ft,Lot Size,Latitude,Longitude,High School,High School Rank,Middle School,Middle School Rank,Elementary School,Elementary School Rank,dollar/sqft,Walk Score
0,2001944,240000.0,2021-03-15,4917 BlueStem DR,80917,1170.0,711.0,38.8976243,-104.737074,Doherty High School,2.0,Russell Middle School,2.0,Carver Elementary School,1.0,205.128205,53
1,8359484,237500.0,2021-03-12,54 Villa DR,81001,2087.0,6509.0,38.8654252,-104.863362,Coronado High School,3.0,Holmes Middle School,3.0,Howbert Elementary School,3.0,113.799713,10
2,7312436,275000.0,2021-03-15,132 Hayden ST,80813,2503.0,9583.0,38.750711,-105.176383,Cripple Creek-Victor Junior-Senior High School,1.0,NaN,NaN,Cresson Elementary School,1.0,109.868158,34
3,1398523,566455.0,2021-03-18,6596 Wolf Gulch DR,80924,4714.0,9088.0,38.9603997,-104.703074,Liberty High School,4.0,Chinook Trail Middle School,NaN,Legacy Peak Elementary School,3.0,120.164404,6
4,4306524,410000.0,2021-03-12,6562 Tillamook DR,80925,1949.0,6600.0,38.8338816,-104.821363,Palmer High School,3.0,NaN,2.0,Coperni 2,NaN,210.364289,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349,4628075,890000.0,2021-04-12,16 Polo DR,80906,3063.0,10890.0,38.7974702,-104.841016,Palmer High School,5.0,NaN,5.0,NaN,NaN,290.564806,NaN
4350,5664354,915000.0,2021-04-12,17925 Black Squirrel RD,80908,4808.0,625086.0,39.092056,-104.627332,Smoky Hill High School,NaN,NaN,NaN,Talbott Steam Innovation School,NaN,190.307820,NaN
4351,1719141,925000.0,2021-04-09,390 Paisley DR,80906,4822.0,13068.0,38.7435755,-104.835224,Mitchell High School,2.0,NaN,NaN,Stratmoor Hills Elementary School,5.0,191.829117,NaN
4352,3250460,990900.0,2021-04-15,4937 Rainbow Gulch TR,80924,6140.0,12120.0,38.9758208,-104.735368,Liberty High School,4.0,NaN,NaN,Pinello Elementary School,5.0,161.384365,NaN


In [10]:
walkdf = df[df["Walk Score"] != "NaN"]
walkdf

,MLS ID,Sale Price,Close Date,Address,Zip Code,Total Sq Ft,Lot Size,Latitude,Longitude,High School,High School Rank,Middle School,Middle School Rank,Elementary School,Elementary School Rank,dollar/sqft,Walk Score
0,2001944,240000.0,2021-03-15,4917 BlueStem DR,80917,1170.0,711.0,38.8976243,-104.737074,Doherty High School,2.0,Russell Middle School,2.0,Carver Elementary School,1.0,205.128205,53
1,8359484,237500.0,2021-03-12,54 Villa DR,81001,2087.0,6509.0,38.8654252,-104.863362,Coronado High School,3.0,Holmes Middle School,3.0,Howbert Elementary School,3.0,113.799713,10
2,7312436,275000.0,2021-03-15,132 Hayden ST,80813,2503.0,9583.0,38.750711,-105.176383,Cripple Creek-Victor Junior-Senior High School,1.0,NaN,NaN,Cresson Elementary School,1.0,109.868158,34
3,1398523,566455.0,2021-03-18,6596 Wolf Gulch DR,80924,4714.0,9088.0,38.9603997,-104.703074,Liberty High School,4.0,Chinook Trail Middle School,NaN,Legacy Peak Elementary School,3.0,120.164404,6
4,4306524,410000.0,2021-03-12,6562 Tillamook DR,80925,1949.0,6600.0,38.8338816,-104.821363,Palmer High School,3.0,NaN,2.0,Coperni 2,NaN,210.364289,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,7444460,1000000.0,2021-03-12,2975 S Electra DR,80906,5364.0,34670.0,38.8229772,-104.875324,NaN,NaN,NaN,3.0,NaN,0.0,186.428039,0
2625,6416567,1165000.0,2021-03-09,4558 Silver Nell DR,80908,4527.0,121097.0,39.083166,-104.744444,NaN,4.0,NaN,4.0,NaN,4.0,257.344820,0
2626,6751431,3800000.0,2021-03-11,755 El Pomar RD #632,80906,5862.0,882.0,38.7894231,-104.854078,NaN,5.0,NaN,5.0,NaN,NaN,648.242921,31
2627,2174596,295000.0,2021-03-17,117 Rose DR,80911,1131.0,6000.0,38.7561206,-104.744096,NaN,1.0,NaN,NaN,NaN,NaN,260.831123,49


In [12]:
df.to_csv("Resources/final.csv", sep=",", encoding='utf-8', index=False, header=True)